<br><font face="Times New Roman" size=5><div dir=ltr align=center>
<font color=blue size=8>
    Introduction to Machine Learning <br>
<font color=red size=5>
    Sharif University of Technology - Computer Engineering Department <br>
    Fall 2022<br> <br>
<font color=black size=6>
    Homework 2: Practical - Decision Tree   
<font color=black size=4>
    Hamidreza Yaghoubi 
    
<br><br>
<font size=4>
In this homework, we are going to implement the Classification Decision Tree. Keep in mind to complete all of the following questions and write your own codes in the TODO cells.

<font face="Times New Roman" size=4><div dir=ltr>
# Problem 2: Classification Decision Tree (100 points)
We will implement a Classification Decision Tree from scratch in the following problem. Then we will use our model to predict malignant and benign breast cancer. For this purpose, we will use the breast_cancer.csv dataset which you can find more details about it <a href="https://www.kaggle.com/datasets/merishnasuwal/breast-cancer-prediction-dataset"><font face="Roboto">here</font></a>.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from math import log
import random

<font face="Times New Roman" size=4><div dir=ltr>
## Classification Decision Tree Class (60 points)
In this section, you only need to fill TODO parts. You can find the logic and formula in both course slides and the web, but fill it out on your own. 

In [ ]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        if self.is_leaf: self.value = value

    def is_leaf(self):
        if ((self.left == None) and (self.right == None)): return True
        else: return False

In [ ]:
class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def is_splitting_finished(self, depth, num_class_labels, num_samples):
        if depth > self.max_depth: return True
        elif num_samples < self.min_samples_split: return True
        elif num_class_labels == len(self.used_features): return True
        else: return False

    def split(self, X, y, feature, threshold):
        mk = X[ :, dict([ [ a, b ] for b, a in enumerate(vlu) ])[ feature ] ] <= threshold
        return X[ mk ], X[ ~mk ], y[ mk ], y[ ~mk ]

    def entropy(self, y):
        if float(np.count_nonzero(y)) == 0: return 0
        else: 
            sum = 0
            x = float(np.count_nonzero(y)) / float(y.size)
            x2 = np.where(x > 0, x, 1e-15)
            sum += np.where(x2 > 1e-14, -x2 * np.log2(x2), 0)
            x = 1 - x 
            x3 = np.where(x > 0, x, 1e-15)
            sum += np.where(x3 > 1e-14, -x3 * np.log2(x3), 0)
            return sum


    def information_gain(self, X, y, feature, threshold):
        if X.size == 0: return 0
        mk = X[ :, dict([ [ a, b ] for b, a in enumerate(vlu) ])[ feature ] ] <= threshold
        result = self.entropy(y)
        result -= (np.count_nonzero(mk) / X.size) * self.entropy(y[ mk ]) + (1 - (np.count_nonzero(mk) / X.size)) * self.entropy(y[ ~mk ])
        return result

    def best_split(self, X, y):
        features = vlu[ np.concatenate((~self.used_features, [ False ])) ]
        np.random.shuffle(features)
        max, ts, fs = 0, 0, vlu[ 0 ]
        for feature in features:
            thresholds = np.unique(X[ :, dict([ [ a, b ] for b, a in enumerate(vlu) ])[ feature ] ])
            for threshold in thresholds:
                goal = self.information_gain(X, y, feature, threshold)
                if goal > max:
                    ts, fs = threshold, feature
                    max = goal
        return fs, ts, max

    def build_tree(self, X, y, depth=0):
        node = Node()
        sum = 0
        rd = np.round(np.sum(y) / y.size)
        for i in range (len(self.used_features)):
            sum += self.used_features[ i ]
        if self.is_splitting_finished(depth, sum, len(X)):
            node.value = rd
            return node
        fs, ts, ig = self.best_split(X, y)
        X_l, X_r, y_l, y_r = self.split(X, y, fs, ts)
        if  (y_l.size == 0) or (y_r.size == 0) or (ig <= 0):
            node.value = rd
            return node
        else:
            node.feature = fs
            node.threshold = ts
            self.used_features[ dict([ [ a, b ] for b, a in enumerate(vlu) ])[ fs ] ] = True
            node.left = self.build_tree(X_l, y_l, depth + 1)
            node.right = self.build_tree(X_r, y_r, depth + 1)
            self.used_features[ dict([ [ a, b ] for b, a in enumerate(vlu) ])[ fs ] ] = False
            return node

    def fit(self, X, y):
        self.used_features = np.array([ False for i in range(len(vlu) - 1) ])
        self.root = self.build_tree(X, y)

    def predict(self, X):
        y = []
        for i in range (len(X)):
            y.append(0)
        for i in range(len(X)):
            node = self.root
            while not node.is_leaf():
                if X[ i, dict([ [ a, b ] for b, a in enumerate(vlu) ])[ node.feature ] ] <= node.threshold: node = node.left
                else: node = node.right
            y[ i ] = node.value
        return np.array(y)

<font face="Times New Roman" size=4><div dir=ltr>
## Data Prepration (20 points)
In this section, you must perform a good EDA for data. Then split it into train and validation data. We will then use the validation data to find the best model hyperparameters.  

In [ ]:
breast_cancer_pdf = pd.read_csv("breast_cancer.csv")

In [ ]:
breast_cancer_pdf.describe()
vlu = breast_cancer_pdf.columns.values
del dict([ [ a, b ] for b, a in enumerate(vlu) ])['diagnosis']
data = breast_cancer_pdf.to_numpy()


In [ ]:
x = data[ :, :5 ]
y = data[ :, 5 ]

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.70, random_state=42)

<font face="Times New Roman" size=4><div dir=ltr>
## Training And Tuning Hyperparameters (20 points)
In this section, you only need to find the best hyperparameters for your model. You can test different values and permutations of hyperparameters by adding them to the lists below. Your model must have at least accuracy=0.85 on validation data.

In [ ]:
max_depths = [2, 3, 4, 5, 6, 7, 8,]
min_samples_splits = [1, 2, 3]

In [ ]:
best_max_depth = None
best_min_samples_split = None
best_accuracy = 0
best_model = None
for max_depth in max_depths:
    for min_samples_split in min_samples_splits:
        clf = DecisionTree(max_depth, min_samples_split)
        clf.fit(x_train, y_train)
        y_val_pred = clf.predict(x_val)
        y_train_pred = clf.predict(x_train)
        accuracy = accuracy_score(y_val_pred, y_val)
        train_accuracy = accuracy_score(y_train_pred, y_train)
        print(f"accuracy of training set for [min_samples_splits={min_samples_split}-max_depths={max_depth}] ={train_accuracy}")
        print(f"accuracy of validation set for [min_samples_splits={min_samples_split}-max_depths={max_depth}] ={accuracy}")
        print("------------------------------------------------")
        if accuracy >= best_accuracy:
            best_accuracy = accuracy
            best_max_depth = max_depth
            best_min_samples_split = min_samples_split
            best_model = clf

accuracy of training set for [min_samples_splits=1-max_depths=2] =0.7647058823529411
accuracy of validation set for [min_samples_splits=1-max_depths=2] =0.8295739348370927
------------------------------------------------
accuracy of training set for [min_samples_splits=2-max_depths=2] =0.7647058823529411
accuracy of validation set for [min_samples_splits=2-max_depths=2] =0.8295739348370927
------------------------------------------------
accuracy of training set for [min_samples_splits=3-max_depths=2] =0.7647058823529411
accuracy of validation set for [min_samples_splits=3-max_depths=2] =0.8295739348370927
------------------------------------------------
accuracy of training set for [min_samples_splits=1-max_depths=3] =0.8
accuracy of validation set for [min_samples_splits=1-max_depths=3] =0.8571428571428571
------------------------------------------------
accuracy of training set for [min_samples_splits=2-max_depths=3] =0.8
accuracy of validation set for [min_samples_splits=2-max_dept